In [3]:
# autoreload
%reload_ext autoreload
%autoreload 2

In [1]:
### imports
# general
import pandas as pd

import re
import googlemaps

In [4]:
### load data
fp = "data/Orlando_data.xlsx"
# algae data
df = pd.read_excel(fp, sheet_name="all_data")
df = df[df.Include == 'yes']    # filter out data not matching criteria
df['Year'] = pd.to_datetime(df['Year'], format='%Y')    # datetime format for later plotting
df[['DOI', 'Year', 'Authors',]] = df[['DOI', 'Year', 'Authors']].ffill()
df.columns = df.columns.str.replace(' ', '_')   # process columns to replace whitespace with underscore
df.columns = df.columns.str.replace('[()]', '', regex=True) # remove '(' and ')' from column names
# remove any rows in which 'n' has '~' in the string
df = df[~df['n'].str.contains('~', na=False)]
df.head()

,Extractor,Notes,Secondary_check,Include,Authors,DOI,Year,Species_types,Taxa,Location,...,Salinity_sd_ppt,n,Calcification,Calcification_se,Calcification_sd,Calcification_units,Duration,Methods,Notes+,Notes++
0,Orlando,NaN,1,yes,Bergstrom E.; Lahnstein J.; Collins H.; Page T...,10.1111/jpy.13290,2023-01-01,Porollithon cf onkodes,Algae,"Lizard Island, GBR, Australia",...,NaN,5,1.970803,0.25555,NaN,mg cm-2day-1,NaN,NaN,NaN,NaN
1,Orlando,NaN,1,yes,Bergstrom E.; Lahnstein J.; Collins H.; Page T...,10.1111/jpy.13290,2023-01-01,Porollithon cf onkodes,Algae,"Lizard Island, GBR, Australia",...,NaN,5,2.408759,0.18976,NaN,mg cm-2day-1,NaN,NaN,NaN,NaN
2,Orlando,NaN,1,yes,Bergstrom E.; Lahnstein J.; Collins H.; Page T...,10.1111/jpy.13290,2023-01-01,Porollithon cf onkodes,Algae,"Lizard Island, GBR, Australia",...,NaN,5,1.664234,0.167864,NaN,mg cm-2day-1,NaN,NaN,NaN,NaN
3,Orlando,NaN,1,yes,Bergstrom E.; Lahnstein J.; Collins H.; Page T...,10.1111/jpy.13290,2023-01-01,Porollithon cf onkodes,Algae,"Lizard Island, GBR, Australia",...,NaN,5,1.839416,0.124074,NaN,mg cm-2day-1,NaN,NaN,NaN,NaN
4,Orlando,NaN,1,yes,Bergstrom E.; Lahnstein J.; Collins H.; Page T...,10.1111/jpy.13290,2023-01-01,Lithophyllum cf insipidum,Algae,"Lizard Island, GBR, Australia",...,NaN,5,1.467626,0.131372,NaN,mg cm-2day-1,NaN,NaN,NaN,NaN


## Cleaning coordinates

In [ ]:
# assign value for each row to coords, or if no coords, location
df['loc'] = df['Cleaned_Coords'].fillna(df['Location'])
df['loc'].unique()

In [ ]:
coords_array = df['Cleaned_Coords'].dropna().unique()
coords_array

In [ ]:
def dms_to_dd(degrees, minutes=0, seconds=0, direction=""):
    """Convert degrees, minutes, and seconds to decimal degrees."""
    decimal_degrees = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
    if direction in ["S", "W"]:
        decimal_degrees *= -1
    return decimal_degrees

def parse_coordinates(coord):
    """Convert various coordinate formats to decimal degrees."""
    coord = coord.strip()
    coord = coord.replace("''", '"')
    coord = coord.replace('"', "+")
    coord = coord.replace("′", "'")
    coord = coord.replace("’", "'")

    # **Decimal Degrees (DD)**
    dd_match = re.match(r"(-?\d+\.\d+)\s*[\u00B0]?\s*,?\s*(-?\d+\.\d+)\s*[\u00B0]?", coord)
    if dd_match:
        return float(dd_match.group(1)), float(dd_match.group(2))

    # **Degrees and Decimal Minutes (DMM)**
    dmm_match = re.match(
        r"(\d+\.?\d*)\u00B0\s*([NS]),?\s*(\d+\.?\d*)\u00B0\s*([EW])", coord
    )
    if dmm_match:
        lat_dd = dms_to_dd(dmm_match.group(1), direction=dmm_match.group(2))
        lon_dd = dms_to_dd(dmm_match.group(3), direction=dmm_match.group(4))
        return lat_dd, lon_dd

    # **Degrees, Minutes, and Seconds (DMS) & Degrees and Minutes (DM)**
    dms_match = re.match(
        r"(\d+)\u00B0\s*(\d+)[′']\s*([\d.]*)?[\″+]?\s*([NSEW])\D+"
        r"(\d+)\u00B0\s*(\d+)[′']\s*([\d.]*)?[\″+]?\s*([EWNS])", coord
    )
    if dms_match:
        lat_dd = dms_to_dd(dms_match.group(1), dms_match.group(2), dms_match.group(3) or 0, dms_match.group(4))
        lon_dd = dms_to_dd(dms_match.group(5), dms_match.group(6), dms_match.group(7) or 0, dms_match.group(8))
        return lat_dd, lon_dd

    raise ValueError(f"Unknown coordinate format: {coord}")

correct = 0
for coord in coords_array:
    try:
        lat, lon = parse_coordinates(coord)
        print(f"Original: {coord} → Decimal Degrees: ({lat}, {lon})")
        correct += 1
    except ValueError as e:
        print(e)
        
print('# CORRECT:', correct)


## Filling missing coordinates with Google Maps API

In [ ]:
# load in others
denisa_df = pd.read_excel("data/Data extraction_Denisa.xlsx", sheet_name="Data extraction")
ashtyn_df = pd.read_excel("data/DataExtraction_AI.xlsx", sheet_name="Data_Extraction")

# concatenate data
GOOGLE_MAPS_API_KEY = utils.read_yaml("api_keys.yaml")['google_maps_api']

gmaps_client = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

locs = pd.DataFrame(denisa_df.Location.unique(), columns=["Location"])

# comment this out to avoid too many api calls
locs[["Latitude", "Longitude"]] = locs["Location"].apply(lambda x: utils.get_coordinates_from_gmaps(x, gmaps_client))


# denisa_df.Location.unique(), ashtyn_df.Location.unique()

In [ ]:
# read locations.yaml
locs = utils.read_yaml("data/locations.yaml")

locs_df = pd.DataFrame([
    {
        "DOI": entry["DOI"],
        "Latitude": entry["coordinates"]["Latitude"],
        "Longitude": entry["coordinates"]["Longitude"],
        "Location": entry["location"]
    }
    for entry in locs
])

# drop nans
locs_df = locs_df.dropna()
# read a list of dictionaries into a single dictionary, with the keys being the location names
# locs = {k: v for d in locs for k, v in d.items()}

# save dictionary to csv
# locs = pd.DataFrame(locs)
# locs.to_csv("data/locations.csv")

## Visualising

In [ ]:
### write locs to yaml, with nice formatting
locs_list = []
for _, row in doi_locs.iterrows():
    locs_list.append({
        'coordinates': {
            'Latitude': row['Latitude'],
            'Longitude': row['Longitude']
        },
        'location': row['Location'],
        'DOI': row['DOI']
    })

utils.write_yaml(locs_list, "data/locations.yaml")

In [ ]:
### plot spatial disribution of studies
locations = read_yaml("data/locations.yaml")

# create figure
fig, ax = plt.subplots(1, 1, figsize=(10, 20), subplot_kw={'projection': ccrs.PlateCarree()}, dpi=300)
ax.set_extent([-180, 180, -40, 40], crs=ccrs.PlateCarree())

# add features
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN, alpha=0.3)
ax.add_feature(cfeature.COASTLINE, edgecolor='lightgray')
ax.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray', alpha=0.2)

# add locations with colors
colors = sns.color_palette("hsv", len(locations))
for i, data in enumerate(locations):
    coords = data["coordinates"]
    ax.plot(coords["Longitude"], coords["Latitude"], 'o', markeredgecolor='darkgrey', markersize=5, color=colors[i], transform=ccrs.PlateCarree(), label=data["location"])

# add legend
ax.legend(loc='lower center', bbox_to_anchor=(0.5, -1.1), ncol=len(locations)//10, fontsize=6);